In [ ]:
using Pkg; Pkg.instantiate()

In [204]:
# import Pkg; Pkg.add("PyCall"); Pkg.add("JLD"); Pkg.add("LinearAlgebra")
using PyCall
using JLD
using LinearAlgebra
using Random
using Base
using Distributed
using Dates
# using UnicodePlots 
using NPZ
using Plots
# Distributed.addprocs(5)

In [4]:
"""First convert numpy indices of atom coordinates
to Julia's form of array."""

py"""
    import numpy as np
    ind_1nn_pbc = np.load("fcc_192/ind_1nn_pbc.npy")
    ind_2nn_pbc = np.load("fcc_192/ind_2nn_pbc.npy")
    ind_3nn_pbc = np.load("fcc_192/ind_3nn_pbc.npy")
    ind_4nn_pbc = np.load("fcc_192/ind_4nn_pbc.npy")
    ind_5nn_pbc = np.load("fcc_192/ind_5nn_pbc.npy")
    ind_6nn_pbc = np.load("fcc_192/ind_6nn_pbc.npy")
    """

ind_1nn_pbc = convert(Matrix{Int64}, PyArray(py"ind_1nn_pbc"o));
ind_2nn_pbc = convert(Matrix{Int64}, PyArray(py"ind_2nn_pbc"o));
ind_3nn_pbc = convert(Matrix{Int64}, PyArray(py"ind_3nn_pbc"o));
ind_4nn_pbc = convert(Matrix{Int64}, PyArray(py"ind_4nn_pbc"o));
ind_5nn_pbc = convert(Matrix{Int64}, PyArray(py"ind_5nn_pbc"o));
ind_6nn_pbc = convert(Matrix{Int64}, PyArray(py"ind_6nn_pbc"o));

In [17]:
cpr(2,2)

6-element Vector{Float64}:
 1.6
 1.2649110640673515
 0.8000000000000002
 0.9999999999999996
 0.6324555320336759
 0.40000000000000013

In [232]:
pth = "fcc_192/ind_pair_pbc.jld"
save(pth, "ind_1nn_pbc", ind_1nn_pbc,
     "ind_2nn_pbc", ind_2nn_pbc,
     "ind_3nn_pbc", ind_3nn_pbc,
     "ind_4nn_pbc", ind_4nn_pbc,
     "ind_5nn_pbc", ind_5nn_pbc,
     "ind_6nn_pbc", ind_6nn_pbc,)

#### The sub func part.

- From 1nn to 6nn pair, the indices are stored in <code>ind_pair.jld</code> file.

In [237]:
pth = "fcc_192/ind_pair_pbc.jld"

#* Suit with the Julia's format
ind_1nn = load(pth, "ind_1nn_pbc") .+1
ind_2nn = load(pth, "ind_2nn_pbc") .+1
ind_3nn = load(pth, "ind_3nn_pbc") .+1
ind_4nn = load(pth, "ind_4nn_pbc") .+1
ind_5nn = load(pth, "ind_5nn_pbc") .+1
ind_6nn = load(pth, "ind_6nn_pbc") .+1

15488×2 Matrix{Int64}:
    1    74
    1   268
    1   666
    1   860
    1  1866
    1  2060
    1  2458
    1  2652
    2    73
    2    75
    ⋮  
 5115  5170
 5115  5172
 5116  5171
 5117  5174
 5118  5173
 5118  5175
 5119  5174
 5119  5176
 5120  5175

#### Second part

Executing the SA for SQS generation.

In [178]:
function main(iter)
    ele_list = ele_list_gen(cr_, mn_, co_, ni_, "randchoice")
    cor_func = cor_func_all(ele_list)

    step_count = 0
    cor_func_raw = cor_func
    while true
        action = [rand(1:192), rand(1:192)]
        ele_list_n, r, cor_func_n, done = swap_step(action, cor_func_raw, ele_list, target_val)
        r_ = exp(r/3)
        if rand() <= min(r_, 1) && cor_func_n != cor_func_raw
            ele_list = ele_list_n
            cor_func_raw = cor_func_n
            step_count += 1
        end

        if done
            println("iter: $iter, step: $step_count") 
            return ele_list_n
            break
        end
    end
end

main (generic function with 1 method)

In [143]:
@everywhere include("fcc_192.jl");
cor_func_list = zeros(Float64, 1000)
vlock = Threads.ReentrantLock()

Threads.@threads for i in 1:1000
    ele_gen = ele_list_gen(0.25, 0.25, 0.25, 0.25)
    # lock(vlock) do
    cor_func_list[i] = cor_func_all(ele_gen, true)
    # end
end

histogram(cor_func_list)

[0.0 0.0 0.0 0.0 0.0 -4.547473508864641e-13; 0.0 -2.2737367544323206e-13 0.0 0.0 0.0 0.0; 0.0 -2.2737367544323206e-13 0.0 0.0 0.0 -9.094947017729282e-13; 0.0 2.2737367544323206e-13 0.0 0.0 0.0 -4.547473508864641e-13; 0.0 -2.2737367544323206e-13 0.0 0.0 0.0 -1.8189894035458565e-12; 0.0 1.1368683772161603e-13 0.0 0.0 0.0 2.2737367544323206e-13]
      From worker 2:	[0.0 0.0 0.0 0.0 0.0 -4.547473508864641e-13; 0.0 -2.2737367544323206e-13 0.0 0.0 0.0 0.0; 0.0 -2.2737367544323206e-13 0.0 0.0 0.0 -9.094947017729282e-13; 0.0 2.2737367544323206e-13 0.0 0.0 0.0 -4.547473508864641e-13; 0.0 -2.2737367544323206e-13 0.0 0.0 0.0 -1.8189894035458565e-12; 0.0 1.1368683772161603e-13 0.0 0.0 0.0 2.2737367544323206e-13]


      From worker 6:	[0.0 0.0 0.0 0.0 0.0 -4.547473508864641e-13; 0.0 -2.2737367544323206e-13 0.0 0.0 0.0 0.0; 0.0 -2.2737367544323206e-13 0.0 0.0 0.0 -9.094947017729282e-13; 0.0 2.2737367544323206e-13 0.0 0.0 0.0 -4.547473508864641e-13; 0.0 -2.2737367544323206e-13 0.0 0.0 0.0 -1.8189894035458565e-12; 0.0 1.1368683772161603e-13 0.0 0.0 0.0 2.2737367544323206e-13]


      From worker 4:	[0.0 0.0 0.0 0.0 0.0 -4.547473508864641e-13; 0.0 -2.2737367544323206e-13 0.0 0.0 0.0 0.0; 0.0 -2.2737367544323206e-13 0.0 0.0 0.0 -9.094947017729282e-13; 0.0 2.2737367544323206e-13 0.0 0.0 0.0 -4.547473508864641e-13; 0.0 -2.2737367544323206e-13 0.0 0.0 0.0 -1.8189894035458565e-12; 0.0 1.1368683772161603e-13 0.0 0.0 0.0 2.2737367544323206e-13]
      From worker 3:	[0.0 0.0 0.0 0.0 0.0 -4.547473508864641e-13; 0.0 -2.2737367544323206e-13 0.0 0.0 0.0 0.0; 0.0 -2.2737367544323206e-13 0.0 0.0 0.0 -9.094947017729282e-13; 0.0 2.2737367544323206e-13 0.0 0.0 0.0 -4.547473508864641e-13; 0.0 -2.2737367544323206e-13 0.0 0.0 0.0 -1.8189894035458565e-12; 0.0 1.1368683772161603e-13 0.0 0.0 0.0 2.2737367544323206e-13]


      From worker 5:	[0.0 0.0 0.0 0.0 0.0 -4.547473508864641e-13; 0.0 -2.2737367544323206e-13 0.0 0.0 0.0 0.0; 0.0 -2.2737367544323206e-13 0.0 0.0 0.0 -9.094947017729282e-13; 0.0 2.2737367544323206e-13 0.0 0.0 0.0 -4.547473508864641e-13; 0.0 -2.2737367544323206e-13 0.0 0.0 0.0 -1.8189894035458565e-12; 0.0 1.1368683772161603e-13 0.0 0.0 0.0 2.2737367544323206e-13]


                    ┌                                        ┐ 
   [2500.0, 3000.0) ┤▌ 4                                       
   [3000.0, 3500.0) ┤██████▍ 54                                
   [3500.0, 4000.0) ┤███████████████████████▏ 198              
   [4000.0, 4500.0) ┤███████████████████████████████████  301  
   [4500.0, 5000.0) ┤█████████████████████████████▌ 253        
   [5000.0, 5500.0) ┤██████████████▎ 122                       
   [5500.0, 6000.0) ┤█████▌ 48                                 
   [6000.0, 6500.0) ┤██▎ 18                                    
   [6500.0, 7000.0) ┤▎ 2                                       
                    └                                        ┘ 
                                     Frequency                 

In [144]:
Distributed.nprocs()

6

In [ ]:
include("fcc_192.jl");
cor_list = main(1)

In [ ]:
@everywhere include("fcc_192.jl");
result = Distributed.pmap(main, 1:200);

In [210]:
time_ = [i[1] for i in result]
step_ = [i[2] for i in result]
histogram(time_)

              ┌                                        ┐ 
   [0.0, 0.5) ┤███████████████████████████████████  143  
   [0.5, 1.0) ┤█████████▊ 40                             
   [1.0, 1.5) ┤██▌ 10                                    
   [1.5, 2.0) ┤█▎ 5                                      
   [2.0, 2.5) ┤  0                                       
   [2.5, 3.0) ┤▌ 2                                       
              └                                        ┘ 
                               Frequency                 

In [211]:
save("Cr25Mn25Co25_try_jltime_pbc.npy", time_)
save("Cr25Mn25Co25_try_jlstep_pbc.npy", step_)